In [334]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from functools import reduce
import operator
from datetime import timedelta, datetime
import datetime

In [335]:
import ahocorasick
import pandas as pd
import operator
from functools import reduce    # reduce the dimension of the array

# step 1: read corresponding data file

In [336]:

time = datetime.datetime.now()
today = str(time.year)+str(time.month)+str(time.day)
datafile = '20201029news_content.csv'
def read_data(datafile):
    try:
        data = pd.read_csv(datafile)
        return data
    except Exception as e:
        print("read failed，please try again\n"+e)

In [337]:
# only keep English news
data = read_data(datafile)
data = data.loc[:62,:]
data

,Unnamed: 0,news source,news content,link
0,0,techcrunch.com,linkedins career explorer helps you identify n...,https://techcrunch.com/2020/10/28/linkedins-ca...
1,1,techcrunch.com,marriott international announces partnership w...,https://techcrunch.com/2020/10/28/marriott-int...
2,2,techcrunch.com,ford will reveal its all electric transit van ...,https://techcrunch.com/2020/10/28/ford-will-re...
3,3,techcrunch.com,section 230 barely rates a mention in senates ...,https://techcrunch.com/2020/10/28/section-230-...
4,4,techcrunch.com,rocket lab successfully launches 10 earth obse...,https://techcrunch.com/2020/10/28/rocket-lab-s...
5,5,techcrunch.com,daily crunch apple search,https://techcrunch.com/2020/10/28/daily-crunch...
6,6,techcrunch.com,we need new business models to burst old media...,https://techcrunch.com/2020/10/28/we-need-new-...
7,7,techcrunch.com,fiat chrysler plans to bring an ram electric p...,https://techcrunch.com/2020/10/28/fiat-chrysle...
8,8,techcrunch.com,us online holiday sales to reach 189b this yea...,https://techcrunch.com/2020/10/28/us-online-ho...
9,9,techcrunch.com,true social app privacy exposed data,https://techcrunch.com/2020/10/28/true-social-...


# step 2: find the topics of the articles

In [338]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english')) 

topics = ['5G', 'AI', 'ARVR', 'carelectronics', 'consumer', 'hardware', 'internet', 'security', 'selfdriving']

In [339]:
def topic_score(text, topic):
    f_en = open('./word dictionaries/words_dict_En_' + topic + '.txt', 'r')
    f_cn = open('./word dictionaries/words_dict_Ch_' + topic + '.txt', 'r')
    dic = dict(dict(eval(f_en.read())[:30]), **dict(eval(f_cn.read())[:30]))
    
    # tokenize the sentence
    word_tokens = word_tokenize(text) 
    # filter stopwords
    filtered_sentence = [w for w in word_tokens if not w in stop_words]
    scores = [dic[w] if w in dic.keys() else 0 for w in filtered_sentence]
    score = sum(scores)
    return score

In [340]:
for topic in topics:
    data[topic + '_title_score'] = 0
    for i in range(data.shape[0]):
    #     print(data.loc[i, 'news content'])
        data.loc[i, topic + '_title_score'] = topic_score(text = data.loc[i, 'news content'], topic = topic)

In [341]:
import requests
from bs4 import BeautifulSoup

def get_article(url, source):
    
    if source in ['techcrunch.com', 'venturebeat.com']:
        r = requests.get(url)
        content = r.text
        soup = BeautifulSoup(content, 'lxml')

        divs = soup.find_all('div', class_ = "article-content")

        ps = list()
        for div in divs:
            ps.extend(div.find_all('p'))

        text = list()
        for p in ps:
            try:
                text.append(p.get_text())
            except:
                pass
        text = ' '.join(text)
        return text
    
    elif source in ['https://www.businessinsider.com/sai']:
        r = requests.get(url)
        content = r.text
        soup = BeautifulSoup(content, 'lxml')

        divs = soup.find_all('div', id = "piano-inline-content-wrapper")

        ps = list()
        for div in divs:
            ps.extend(div.find_all('p'))

        text = list()
        for p in ps:
            try:
                text.append(p.get_text())
            except:
                pass
        text = ' '.join(text)
        return text
    else:
        return ''

In [342]:
data['article_content'] = 0
for i in range(data.shape[0]):
    data.loc[i, 'article_content'] = get_article(url = data.loc[i, 'link'], source = data.loc[i, 'news source'])

In [343]:
data[['news source','article_content']].head(5)

,news source,article_content
0,techcrunch.com,One of the key side-effects of the Covid-19 pa...
1,techcrunch.com,The COVID-19 pandemic has hit the hospitality ...
2,techcrunch.com,Ford plans to reveal in November an all-electr...
3,techcrunch.com,Today’s Senate hearing on immensely important ...
4,techcrunch.com,New Zealand launch provider Rocket Lab has pu...


In [344]:
for topic in topics:
    data[topic + '_content_score'] = 0
    for i in range(data.shape[0]):
    #     print(data.loc[i, 'news content'])
        data.loc[i, topic + '_content_score'] = topic_score(text = data.loc[i, 'article_content'], topic = topic)

In [345]:
tag1 = data.loc[[0],['5G_title_score','AI_title_score', 'ARVR_title_score', 'carelectronics_title_score',\
             'consumer_title_score', 'hardware_title_score', 'internet_title_score',\
             'security_title_score', 'selfdriving_title_score']].T[0].idxmax()

In [346]:
tag2 = data.loc[[0],['5G_content_score', 'AI_content_score', 'ARVR_content_score','carelectronics_content_score', \
                     'consumer_content_score', 'hardware_content_score', 'internet_content_score',\
                     'security_content_score', 'selfdriving_content_score']].T[0].idxmax()

# step 3: tag the articles

In [347]:
data['tag1'] = 0
data['tag2'] = 0
for i in range(data.shape[0]):
    tag1 = data.loc[[i],['5G_title_score','AI_title_score', 'ARVR_title_score', 'carelectronics_title_score',\
             'consumer_title_score', 'hardware_title_score', 'internet_title_score',\
             'security_title_score', 'selfdriving_title_score']].T[i].idxmax()
    tag2 = data.loc[[i],['5G_content_score', 'AI_content_score', 'ARVR_content_score','carelectronics_content_score', \
                     'consumer_content_score', 'hardware_content_score', 'internet_content_score',\
                     'security_content_score', 'selfdriving_content_score']].T[i].idxmax()
    
    data.loc[i, 'tag1'] = tag1.split('_')[0]
    data.loc[i, 'tag2'] = tag2.split('_')[0]

In [348]:
data[['news source', 'news content', 'link', 'tag1', 'tag2']]

,news source,news content,link,tag1,tag2
0,techcrunch.com,linkedins career explorer helps you identify n...,https://techcrunch.com/2020/10/28/linkedins-ca...,internet,internet
1,techcrunch.com,marriott international announces partnership w...,https://techcrunch.com/2020/10/28/marriott-int...,5G,internet
2,techcrunch.com,ford will reveal its all electric transit van ...,https://techcrunch.com/2020/10/28/ford-will-re...,carelectronics,internet
3,techcrunch.com,section 230 barely rates a mention in senates ...,https://techcrunch.com/2020/10/28/section-230-...,internet,internet
4,techcrunch.com,rocket lab successfully launches 10 earth obse...,https://techcrunch.com/2020/10/28/rocket-lab-s...,security,internet
5,techcrunch.com,daily crunch apple search,https://techcrunch.com/2020/10/28/daily-crunch...,security,internet
6,techcrunch.com,we need new business models to burst old media...,https://techcrunch.com/2020/10/28/we-need-new-...,internet,internet
7,techcrunch.com,fiat chrysler plans to bring an ram electric p...,https://techcrunch.com/2020/10/28/fiat-chrysle...,consumer,internet
8,techcrunch.com,us online holiday sales to reach 189b this yea...,https://techcrunch.com/2020/10/28/us-online-ho...,security,internet
9,techcrunch.com,true social app privacy exposed data,https://techcrunch.com/2020/10/28/true-social-...,security,internet


# step 4: get the summary of each article

In [349]:
from newspaper import Article

In [350]:
def get_summary(url):
    article = Article(url)
    article.download()
    article.parse()
    article.nlp()
    return article.summary

In [351]:
data['summary'] = data.link.apply(lambda x: get_summary(x))

In [352]:
data = data[data['article_content'] != '']

In [353]:
data = data[['news source', 'news content', 'link', 'tag1', 'tag2','summary']]

# step 5: generate our daily newsletter content

In [354]:
data.head(2)

,news source,news content,link,tag1,tag2,summary
0,techcrunch.com,linkedins career explorer helps you identify n...,https://techcrunch.com/2020/10/28/linkedins-ca...,internet,internet,"To meet some of that challenge, today, LinkedI..."
1,techcrunch.com,marriott international announces partnership w...,https://techcrunch.com/2020/10/28/marriott-int...,5G,internet,The COVID-19 pandemic has hit the hospitality ...


In [355]:
out = ''
for i in range(data.shape[0]):
    out = out + 'Title: ' + data.iloc[i, 1] + '\n'
    out = out + 'Source: ' + data.iloc[i, 0] + '\n'
    out = out + 'Link: ' + data.iloc[i, 2] + '\n'
    if data.iloc[i, 4] == data.iloc[i, 3]:
        out = out + 'Tags: ' + data.iloc[i, 4] + '\n'
    else:
        out = out + 'Tags: ' + data.iloc[i, 4] + ', ' + data.iloc[i, 3] + '\n'
    out = out + 'Summary: ' + data.iloc[i, 5].replace('\n',' ') + '\n'
    out = out + '\n' + '\n'+ '\n'
      

In [356]:
out = out.replace('\xa3','')
out = out.replace('\u2122','')

In [357]:
text_file = open(today + '_news.txt','wt')
text_file.write(out)
text_file.close()

In [358]:
print(out)

Title: linkedins career explorer helps you identify new kinds of jobs based on the skills you have
Source: techcrunch.com
Link: https://techcrunch.com/2020/10/28/linkedins-career-explorer-helps-you-identify-new-kinds-of-jobs-based-on-the-skills-you-have/
Tags: internet
Summary: To meet some of that challenge, today, LinkedIn is launching a new Career Explorer tool to help people find new jobs. The Career Explorer tool is perhaps the most interesting of the new features. It notes that a food server and a customer service specialist (an in-demand job) have a 71% skills overlap. LinkedIn said that to date, around 13 million people have used the those tools to learn new skills for the most in-demand jobs. It helps keep that jobs business evolving at a time when it otherwise might feel somewhat stagnant.



Title: marriott international announces partnership with grab in six southeast asian countries
Source: techcrunch.com
Link: https://techcrunch.com/2020/10/28/marriott-international-annou